<a href="https://colab.research.google.com/github/QuantAnalyticsTorch/quant_analytics_torch/blob/main/examples/RandomPerformanceTensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#@title Upgrade to TensorFlow nightly
!pip uninstall tf-nightly-gpu

Uninstalling tf-nightly-gpu-2.5.0.dev20201128:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tf_nightly_gpu-2.5.0.dev20201128.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/tensorflow/core/protobuf/data/__init__.py
    /usr/local/lib/python3.6/dist-packages/tensorflow/core/protobuf/data/experimental/__init__.py
    /usr/local/lib/python3.6/dist-packages/tensorflow/core/protobuf/data/experimental/snapshot_pb2.py
    /usr/local/lib/python3.6/dist-packages/tensorflow/examples/__init__.py
    /usr/local/lib/python3.6/dist-packages/tensorflow/examples/saved_model/__init__.py
    /usr/local/lib/python3.6/dist-packages/tensorf

In [25]:
import tensorflow as tf

In [26]:
from scipy.special import erf
import numpy as np
import time

In [27]:
x = tf.math.sobol_sample(10,10)

In [28]:
import numpy as np

In [29]:
import tensorflow as tf

In [30]:
import numpy as np
import math

In [31]:
number_time_steps = 128
number = 3

left_index = np.zeros(number_time_steps, dtype=int)
right_index = np.zeros(number_time_steps, dtype=int)
bridge_index = np.zeros(number_time_steps, dtype=int)
left_weight = np.zeros(number_time_steps)
right_weight = np.zeros(number_time_steps)
std_dev = np.zeros(number_time_steps)

_map = np.zeros(number_time_steps, dtype=int)

_map[-1] = 1
bridge_index[0] = number_time_steps - 1
std_dev[0] = math.sqrt(1.0 * number_time_steps)
left_weight[0] = 0
right_weight[0] = 0

j=0
for i in range(1,number_time_steps):
  while _map[j] == True:
    j = j + 1
  k = j
  while _map[k] == False:
    k = k + 1
l = j+((k-1-j)>>1)
_map[l]=i
bridge_index[i]=l
left_index[i]=j
right_index[i]=k
left_weight[i]=(k-l)/(k+1-j)
right_weight[i]=(1+l-j)/(k+1-j)
std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
j=k+1
if j>=number_time_steps:
  j=0

bridge_index_t = tf.constant(tf.convert_to_tensor(bridge_index,dtype=tf.int32))
right_index_t = tf.constant(tf.convert_to_tensor(right_index,dtype=tf.int32))
left_index_t = tf.constant(tf.convert_to_tensor(left_index,dtype=tf.int32))
std_dev_t = tf.constant(tf.convert_to_tensor(std_dev))
right_weight_t = tf.constant(tf.convert_to_tensor(right_weight))
left_weight_t = tf.constant(tf.convert_to_tensor(left_weight))

In [32]:
@tf.function(jit_compile=True, input_signature=(tf.TensorSpec(shape=(None,None), dtype=tf.float64),))
def buildP(path):
  pt = tf.unstack(path)
  return pt

In [33]:
@tf.function(jit_compile=True, input_signature=(tf.TensorSpec(shape=(None,None), dtype=tf.float64),
                                                tf.TensorSpec(shape=(), dtype=tf.int32),
                                                tf.TensorSpec(shape=(), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.float64),
                                                tf.TensorSpec(shape=(None), dtype=tf.float64),
                                                tf.TensorSpec(shape=(None), dtype=tf.float64)))
def buildPath(z, number, number_time_steps, left_index, right_index, bridge_index, left_weight, right_weight, std_dev):
  print(number)
  print(number_time_steps)
  #path = tf.zeros(shape=(number_time_steps,number),dtype=tf.dtypes.float64)
  path = tf.TensorArray(dtype=tf.float64,size=number_time_steps)
  #path = [[0] * number_time_steps] * number
  #print(path)
  #path_t = tf.unstack(path,num=number_time_steps)
  #print(left_index[number-1])
  #print(z)
  #return z[left_index[number-1]]
#def buildPath(z, number, number_time_steps, left_index_t, right_index_t, bridge_index_t, left_weight, right_weight, std_dev):

  #path = tf.unstack(tf.zeros(shape=(number_time_steps,number),dtype=tf.dtypes.float64))
  path = path.write(number_time_steps-1,std_dev[0]*z[0]);
  j = 0
  k = 0
  l = 0
  i = 0
  for i in range(1,number_time_steps):
#    print("i")
#    print(i)
    j = left_index[i]
    k = right_index[i]
    l = bridge_index[i]
#    print("l")
#    print(l)
#    print("k")
#    print(k)
#    print("right_weight")
#    print(right_weight[i])
    path = path.write(l,right_weight[i] * path.read(k) + std_dev[i] * z[i])
    #path[l] = 

  return path.stack()

In [34]:
print(buildPath.pretty_printed_concrete_signatures())

Tensor("number:0", shape=(), dtype=int32)
Tensor("number_time_steps:0", shape=(), dtype=int32)
buildPath(z, number, number_time_steps, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)
  Args:
    z: float64 Tensor, shape=(None, None)
    number: int32 Tensor, shape=()
    number_time_steps: int32 Tensor, shape=()
    left_index: int32 Tensor, shape=<unknown>
    right_index: int32 Tensor, shape=<unknown>
    bridge_index: int32 Tensor, shape=<unknown>
    left_weight: float64 Tensor, shape=<unknown>
    right_weight: float64 Tensor, shape=<unknown>
    std_dev: float64 Tensor, shape=<unknown>
  Returns:
    float64 Tensor, shape=<unknown>


In [35]:
x = tf.math.sobol_sample(128,5,dtype=tf.dtypes.float64)
z = tf.transpose(tf.math.erf(x))

In [36]:
#path = buildPath(z, 5, 8, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)
path = buildPath(z, tf.constant(5), tf.constant(128), left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)

In [37]:
print(buildPath.pretty_printed_concrete_signatures())

buildPath(z, number, number_time_steps, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)
  Args:
    z: float64 Tensor, shape=(None, None)
    number: int32 Tensor, shape=()
    number_time_steps: int32 Tensor, shape=()
    left_index: int32 Tensor, shape=<unknown>
    right_index: int32 Tensor, shape=<unknown>
    bridge_index: int32 Tensor, shape=<unknown>
    left_weight: float64 Tensor, shape=<unknown>
    right_weight: float64 Tensor, shape=<unknown>
    std_dev: float64 Tensor, shape=<unknown>
  Returns:
    float64 Tensor, shape=<unknown>


In [38]:
print(tf.autograph.to_code(buildPath.python_function))

def tf__buildPath(z, number, number_time_steps, left_index, right_index, bridge_index, left_weight, right_weight, std_dev):
    with ag__.FunctionScope('buildPath', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        ag__.ld(print)(ag__.ld(number))
        ag__.ld(print)(ag__.ld(number_time_steps))
        path = ag__.converted_call(ag__.ld(tf).TensorArray, (), dict(dtype=ag__.ld(tf).float64, size=ag__.ld(number_time_steps)), fscope)
        path = ag__.converted_call(ag__.ld(path).write, ((ag__.ld(number_time_steps) - 1), (ag__.ld(std_dev)[0] * ag__.ld(z)[0])), None, fscope)
        j = 0
        k = 0
        l = 0
        i = 0

        def get_state():
            return (path,)

        def set_state(vars_):
            nonlocal path
            (path,) = vars_

        def loop_body(itr):
            nonlocal path
   

In [43]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  128#@param {type:"integer"}
number =  200000#@param {type:"integer"}

# First run (includes graph optimization time)
time_start = time.time()
with tf.device("/gpu:0"):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    z = tf.transpose(tf.math.erf(x))
    w = buildPath(z, number, dim, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)
time_end = time.time()
time_price_cpu = time_end - time_start
print("First time on a CPU: ", time_price_cpu)

# Second run (excludes graph optimization time)
time_start = time.time()
with tf.device("/gpu:0"):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    #print(x)
    z = tf.transpose(tf.math.erf(x))
    w = buildPath(z, number, dim, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)
time_end = time.time()

time_price_cpu = time_end - time_start
print("Second time on a CPU: ", time_price_cpu)

First time on a CPU:  0.3714601993560791
Second time on a CPU:  0.3588728904724121


In [ ]:
import torch

In [ ]:
@torch.jit.script
def foo(x, number: int):
  print(number)
  z = 0.
  for i in range(0,number):
    tmp = x[i]
    z = z + tmp

# See the compiled graph as Python code
print(foo.code)

def foo(x: Tensor,
    number: int) -> None:
  print(number)
  z = 0.
  for _0 in range(torch.__range_length(0, number, 1)):
    i = torch.__derive_index(_0, 0, 1)
    tmp = torch.select(x, 0, i)
    z0 = torch.add(tmp, z, 1)
    z = annotate(float, z0)
  return None



In [ ]:
# Call the function using the TorchScript interpreter
x = torch.zeros(11)

foo(x, 10)

10


In [ ]:
@torch.jit.script
def buildPath(z, number: int, number_time_steps: int, left_index, right_index, bridge_index, left_weight, right_weight, std_dev):
  path = torch.zeros(size=(number_time_steps,number))
  path[-1] = std_dev[0]*z[0]
  j = 0
  k = 0
  l = 0
  i = 0
  for i in range(1,number_time_steps):
#    print("i")
#    print(i)
    j = left_index[i]
    k = right_index[i]
    l = bridge_index[i]
#    print("l")
#    print(l)
#    print("k")
#    print(k)
#    print("right_weight")
#    print(right_weight[i])
    path[l] = right_weight[i] * path[k] + std_dev[i] * z[i]

  return path

In [ ]:
print(buildPath.code)

def buildPath(z: Tensor,
    number: int,
    number_time_steps: int,
    left_index: Tensor,
    right_index: Tensor,
    bridge_index: Tensor,
    left_weight: Tensor,
    right_weight: Tensor,
    std_dev: Tensor) -> Tensor:
  path = torch.zeros([number_time_steps, number], dtype=None, layout=None, device=None, pin_memory=None)
  _0 = torch.mul(torch.select(std_dev, 0, 0), torch.select(z, 0, 0))
  _1 = torch.copy_(torch.select(path, 0, -1), _0, False)
  _2 = torch.__range_length(1, number_time_steps, 1)
  for _3 in range(_2):
    i = torch.__derive_index(_3, 1, 1)
    k = torch.select(right_index, 0, i)
    _4 = annotate(int, k)
    l = torch.select(bridge_index, 0, i)
    _5 = annotate(int, l)
    _6 = torch.mul(torch.select(right_weight, 0, i), torch.select(path, 0, _4))
    _7 = torch.mul(torch.select(std_dev, 0, i), torch.select(z, 0, i))
    _8 = torch.add(_6, _7, alpha=1)
    _9 = torch.copy_(torch.select(path, 0, _5), _8, False)
  return path



In [ ]:
x = torch.randn(256,2)

left_index_t = torch.tensor(left_index)
right_index_t = torch.tensor(right_index)
bridge_index_t = torch.tensor(bridge_index)
left_weight_t = torch.tensor(left_weight)
right_weight_t = torch.tensor(right_weight)
std_dev_t = torch.tensor(std_dev)

path = buildPath(x, 2, 256, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)

In [ ]:
import time

In [ ]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  256#@param {type:"integer"}
number =  100000#@param {type:"integer"}

# First run (includes graph optimization time)
time_start = time.time()
x = torch.randn(dim, number)
path = buildPath(x, number, dim, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)
time_end = time.time()
time_price_cpu = time_end - time_start
print("First time on a CPU: ", time_price_cpu)

# Second run (excludes graph optimization time)
time_start = time.time()
x = torch.randn(dim, number)
path = buildPath(x, number, dim, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)
time_end = time.time()

time_price_cpu = time_end - time_start
print("Second time on a CPU: ", time_price_cpu)

First time on a CPU:  0.40483903884887695
Second time on a CPU:  0.3911397457122803


In [19]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Please install GPU version of TF


In [1]:
import tensorflow as tf

In [9]:
import os
import tensorflow_datasets as tfds

In [11]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.53.234.50:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.53.234.50:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU')]


In [2]:
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
with tf.profiler.experimental.Profile("/logs/profile"):
  c = tf.matmul(a, b)
print("c device: ", c.device)
print(c)

c device:  /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
%load_ext tensorboard

In [5]:
%tensorboard --logdir logs/profile

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2020-11-28 20:38:16.767182: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-11-28 20:38:16.767239: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/main.py", line 75, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 300, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "/usr/loca